In [26]:
import ccxt
import credentials

exchange = ccxt.binanceusdm({
    'enableRateLimit': True,
    'apiKey': credentials.api_key,
    'secret': credentials.api_secret
})

# help = exchange.has
# print(help)
symbol = ['BTC/USDT', 'ETH/USDT']
# balance = exchange.fetchBalance()['info']['assets']
# [print(balance)]

# ticker = exchange.fetchTicker((symbol))
# ask_price = ticker['close']
# balance = exchange.fetchBalance()['total']['USDT']
# buy_quantity = balance / ask_price
# buy = exchange.createMarketOrder(symbol, side = 'BUY', amount = buy_quantity)  # Specify market type as futures
# print(buy)

order = exchange.fetchPositions(symbols=symbol)
print(order)

"""
{'publicAPI': True, 'privateAPI': True, 'CORS': None, 'spot': False, 'margin': False, 'swap': True, 'future': True, 'option': None, 'addMargin': True, 'cancelAllOrders': True, 'cancelOrder': True, 'cancelOrders': None, 'createDepositAddress': False, 'createLimitOrder': True, 'createMarketOrder': True, 'createOrder': True, 'createPostOnlyOrder': True, 'createReduceOnlyOrder': True, 'createStopOrder': True, 'createStopLimitOrder': True, 'createStopMarketOrder': True, 'editOrder': True, 'fetchAccounts': None, 'fetchBalance': True, 'fetchBidsAsks': True, 'fetchBorrowInterest': True, 'fetchBorrowRate': True, 'fetchBorrowRateHistory': True, 'fetchBorrowRatesPerSymbol': False, 'fetchBorrowRates': False, 'fetchCanceledOrders': 'emulated', 'fetchClosedOrder': False, 'fetchClosedOrders': 'emulated', 'fetchCurrencies': True, 'fetchDeposit': False, 'fetchDepositAddress': True, 'fetchDepositAddresses': False, 'fetchDepositAddressesByNetwork': False, 'fetchDeposits': True, 'fetchFundingFee': None, 'fetchFundingFees': None, 'fetchFundingHistory': True, 'fetchFundingRate': True, 'fetchFundingRateHistory': True, 'fetchFundingRates': True, 'fetchIndexOHLCV': True, 'fetchL2OrderBook': True, 'fetchLedger': True, 'fetchLedgerEntry': None, 'fetchLeverageTiers': True, 'fetchMarketLeverageTiers': 'emulated', 'fetchMarkets': True, 'fetchMarkOHLCV': True, 'fetchMyTrades': True, 'fetchOHLCV': True, 'fetchOpenOrder': False, 'fetchOpenOrders': True, 'fetchOrder': True, 'fetchOrderBook': True, 'fetchOrderBooks': False, 'fetchOrders': True, 'fetchOrderTrades': True, 'fetchPermissions': None, 'fetchPosition': True, 'fetchPositions': True, 'fetchPositionsRisk': True, 'fetchPremiumIndexOHLCV': False, 'fetchStatus': True, 'fetchTicker': True, 'fetchTickers': True, 'fetchTime': True, 'fetchTrades': True, 'fetchTradingFee': True, 'fetchTradingFees': True, 'fetchTradingLimits': None, 'fetchTransactions': False, 'fetchTransfers': True, 'fetchWithdrawal': False, 'fetchWithdrawals': True, 'reduceMargin': True, 'setLeverage': True, 'setMargin': False, 'setMarginMode': True, 'setPositionMode': True, 'signIn': False, 'transfer': True, 'withdraw': True, 'borrowMargin': True, 'fetchBorrowRateHistories': False, 'fetchDepositWithdrawFee': 'emulated', 'fetchDepositWithdrawFees': True, 'fetchL3OrderBook': None, 'fetchLastPrices': True, 'fetchLeverage': False, 'fetchOpenInterest': True, 'fetchOpenInterestHistory': True, 'fetchSettlementHistory': True, 'fetchTransactionFee': None, 'fetchTransactionFees': True, 'fetchWithdrawalWhitelist': False, 'repayMargin': True}
"""

[{'info': {'symbol': 'ETHUSDT', 'positionAmt': '0.000', 'entryPrice': '0.0', 'markPrice': '1872.72583939', 'unRealizedProfit': '0.00000000', 'liquidationPrice': '0', 'leverage': '1', 'maxNotionalValue': '3.0E8', 'marginType': 'cross', 'isolatedMargin': '0.00000000', 'isAutoAddMargin': 'false', 'positionSide': 'BOTH', 'notional': '0', 'isolatedWallet': '0', 'updateTime': '1685646019773'}, 'id': None, 'symbol': 'ETH/USDT:USDT', 'contracts': 0.0, 'contractSize': 1.0, 'unrealizedPnl': 0.0, 'leverage': 1.0, 'liquidationPrice': None, 'collateral': 0.0, 'notional': 0.0, 'markPrice': 1872.72583939, 'entryPrice': 0.0, 'timestamp': 1685646019773, 'initialMargin': 0.0, 'initialMarginPercentage': 1.0, 'maintenanceMargin': 0.0, 'maintenanceMarginPercentage': 0.005, 'marginRatio': None, 'datetime': '2023-06-01T19:00:19.773Z', 'marginMode': 'cross', 'marginType': 'cross', 'side': None, 'hedged': False, 'percentage': None}, {'info': {'symbol': 'BTCUSDT', 'positionAmt': '0.000', 'entryPrice': '0.0', 'm

"\n{'publicAPI': True, 'privateAPI': True, 'CORS': None, 'spot': False, 'margin': False, 'swap': True, 'future': True, 'option': None, 'addMargin': True, 'cancelAllOrders': True, 'cancelOrder': True, 'cancelOrders': None, 'createDepositAddress': False, 'createLimitOrder': True, 'createMarketOrder': True, 'createOrder': True, 'createPostOnlyOrder': True, 'createReduceOnlyOrder': True, 'createStopOrder': True, 'createStopLimitOrder': True, 'createStopMarketOrder': True, 'editOrder': True, 'fetchAccounts': None, 'fetchBalance': True, 'fetchBidsAsks': True, 'fetchBorrowInterest': True, 'fetchBorrowRate': True, 'fetchBorrowRateHistory': True, 'fetchBorrowRatesPerSymbol': False, 'fetchBorrowRates': False, 'fetchCanceledOrders': 'emulated', 'fetchClosedOrder': False, 'fetchClosedOrders': 'emulated', 'fetchCurrencies': True, 'fetchDeposit': False, 'fetchDepositAddress': True, 'fetchDepositAddresses': False, 'fetchDepositAddressesByNetwork': False, 'fetchDeposits': True, 'fetchFundingFee': None